In [ ]:
import requests
import datetime
import time
import pymysql
from datetime import datetime

In [ ]:
# Définition d'une fonction de tranformation automatique des timestamps en date :

def get_timestamp(date_str):    # Convertit une date en timestamp (millisecondes)
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    return int(dt.timestamp() * 1000)

In [ ]:
# Création d'une liste de dates en granularité mensuelle pour l'implémenter dans les requêtes (à modifier à patir des imports en minutes) :

start_date = datetime(2017, 1, 1)
end_date = datetime(2025, 6, 1) # La end_date est exclue du rapatriement des données -> 2025/6/1 implique que la dernière date sera 2025/5/31.

date_list = []
current_date = start_date

while current_date <= end_date:
    date_list.append(current_date.strftime("%Y-%m-%d")) # A PARTIR DE L4ECHERLLE HEURE, MODIFIER current_date.strftime("%Y-%m-%d") PAR current_date.strftime("%Y-%m-%d %H:%M:%S")
    next_month = current_date.month + 1
    next_year = current_date.year + (next_month // 12)
    next_month = next_month % 12 or 12  # Gère le passage à décembre
    current_date = datetime(next_year, next_month, 1)


In [ ]:
# Récupération données en jour :

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="RENSEIGNEZ VOTRE MOT DE PASSE MySQL*",
    database="RENSEIGNEZ LE NOM DE VOTRE BASE DE DONNEES"
        )
cursor = conn.cursor()

for i in range(len(date_list) - 1):
    url = f"https://api.binance.com/api/v3/klines?symbol=BTCUSDC&interval=1d&startTime={get_timestamp(date_list[i])}&endTime={get_timestamp(date_list[i+1])}"
    response = requests.get(url)
    data = response.json()
    
    if data:
        values = []   
        for day in data:
            date = datetime.fromtimestamp(day[0] / 1000).strftime('%Y-%m-%d') # FORMAT stftime() à modifier à partir de l'échelle HEURE pour intégrer les heures, minutes et secondes ("%Y-%m-%d %H:%M:%S")
            open_price, high_price, low_price, close_price, volume = day[1], day[2], day[3], day[4], day[5]
            values.append((date, open_price, high_price, low_price, close_price, volume))
            
        cursor.executemany("""
            INSERT INTO #RENSEIGNEZ LE NOM DE LA TABLE DE DESTINATION# (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            """, values)
        conn.commit()
        
    time.sleep(2)

cursor.close()
conn.close()

print("Mise à jour terminée !")

##  Code pour récupérer les données à partir de l'échelle MINUTE

In [ ]:
from datetime import datetime, timedelta

In [ ]:
def get_timestamp1(date_str):    # Convertit une date en timestamp (millisecondes)
    dt = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int(dt.timestamp() * 1000)

In [ ]:
start_date = datetime(2017, 1, 1, 0, 0, 0)
end_date = datetime(2025, 6, 16, 0, 0, 0)

date_list = []
current_date = start_date

while current_date <= end_date:
    date_list.append(current_date.strftime("%Y-%m-%d %H:%M:%S"))  # Ajout avec heures et secondes
    current_date += timedelta(hours=1)  # Step de 1h

In [ ]:
conn = pymysql.connect(
    host="localhost",
    user="RENSEIGNEZ VOTRE NOM D'UTILISATEUR MySQL",
    password="RENSEIGNEZ VOTRE MOT DE PASSE MySQL",
    database="RENSEIGNEZ LE NOM DE VOTRE BASE DE DONNEES"
        )
cursor = conn.cursor()

for i in range(len(date_list) - 1):
    url = f"https://api.binance.com/api/v3/klines?symbol=BTCUSDC&interval=1m&startTime={get_timestamp1(date_list[i])}&endTime={get_timestamp1(date_list[i+1])}"
    response = requests.get(url)
    data = response.json()
    
    if data:
        values = []   
        for day in data:
            date = datetime.fromtimestamp(day[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            open_price, high_price, low_price, close_price, volume = day[1], day[2], day[3], day[4], day[5]
            values.append((date, open_price, high_price, low_price, close_price, volume))
            
        cursor.executemany("""
            INSERT INTO #RENSEIGNEZ LE NOM DE LA TABLE DE DESTINATION# (date, open, high, low, close, volume)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE open=VALUES(open), high=VALUES(high), low=VALUES(low), close=VALUES(close), volume=VALUES(volume);
            """, values)
        conn.commit()
        
    time.sleep(2)

cursor.close()
conn.close()

print("Mise à jour terminée !")